In [1]:
import pandas as pd
import requests

In [2]:
url="https://api.census.gov/data/2019/pep/charagegroups?"


In [16]:
query_url=url+ "get=NAME,POP&for=county:*&in=state:12&AGEGROUP=026"
results_cols=["County, State","Population","Age Group","State Code","County Code"]

In [17]:
results=requests.get(query_url).json()

In [18]:
results

[['NAME', 'POP', 'AGEGROUP', 'state', 'county'],
 ['Washington County, Florida', '4380', '26', '12', '133'],
 ['Duval County, Florida', '138594', '26', '12', '031'],
 ['Bradford County, Florida', '5064', '26', '12', '007'],
 ['Brevard County, Florida', '145752', '26', '12', '009'],
 ['Clay County, Florida', '35844', '26', '12', '019'],
 ['Lafayette County, Florida', '1455', '26', '12', '067'],
 ['Lake County, Florida', '99386', '26', '12', '069'],
 ['Nassau County, Florida', '20231', '26', '12', '089'],
 ['Pinellas County, Florida', '247630', '26', '12', '103'],
 ['Polk County, Florida', '148168', '26', '12', '105'],
 ['St. Lucie County, Florida', '80928', '26', '12', '111'],
 ['Glades County, Florida', '3808', '26', '12', '043'],
 ['Hendry County, Florida', '5779', '26', '12', '051'],
 ['Indian River County, Florida', '54133', '26', '12', '061'],
 ['Jackson County, Florida', '9570', '26', '12', '063'],
 ['Palm Beach County, Florida', '364544', '26', '12', '099'],
 ['St. Johns County, 

In [21]:
df=pd.DataFrame(results,columns=results_cols)
df. columns=df. iloc[0]
df=df.tail(-1)
df.head()

,NAME,POP,AGEGROUP,state,county
1,"Washington County, Florida",4380,26,12,133
2,"Duval County, Florida",138594,26,12,031
3,"Bradford County, Florida",5064,26,12,007
4,"Brevard County, Florida",145752,26,12,009
5,"Clay County, Florida",35844,26,12,019
